
### Extrapolation with text-davinci-002 (SCAN)

In [32]:
!pip install --upgrade openai

     |████████████████████████████████| 42 kB 1.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 162 kB 14.7 MB/s 
  Created wheel for openai: filename=openai-0.18.1-py3-none-any.whl size=53168 sha256=0b5ca78484b355e5e35ec4ec2a0229a014015e73d8acb45fa918cae9b6c56730
  Stored in directory: /root/.cache/pip/wheels/5a/bf/24/fcdc9d2b81f9c7e565bb2036ec9f7cc930056b829895b3bf48
Successfully built openai


## utils

In [1]:
%env OPENAI_API_KEY=

env: OPENAI_API_KEY=


In [3]:
import pandas as pd
import itertools
import json

def save_results(prompt, ground_truth, responses_, file_name):
  
  results = {'prompt':[], 'ground_truth':[],'candidate':[]}
    
  for p, gt, resp in zip(prompt, ground_truth, responses_):
    results['prompt'].append(p)
    results['ground_truth'].append(gt)
    results['candidate'].append(resp['choices'][0]['text'])

  #save results
  df = pd.DataFrame(results)
  df.to_csv(file_name, sep='\t', index=None, header= False)  

## Prompt

In [33]:
base_prompt="Example 1:\nInstruction: jump twice then turn left twice\nExplanation:\nBecause of the word \"then\", the instruction can be divided in two sub-instructions: 1) \"jump twice\" and 2) \"turn left twice\"\n1) \"jump twice\" results in 2 JUMP commands.\n2) \"turn left twice\" results in 2 LEFT commands.\nOutput: JUMP 1 JUMP 2 || LEFT 1 LEFT 2\n\nExample 2:\nInstruction: turn opposite right after run thrice\nExplanation:\nThe word \"after\", results in the inversion of the two sub-instructions 2) \"turn opposite right\" and 1) \"run thrice\".\n1) \"run thrice\" results in 3 RUN commands.\n2) \"turn opposite right\" . The word \"opposite\", results in 2 repetitions of command RIGHT, therefore, \"turn opposite right\" results in 1 RIGHT RIGHT command.\nOutput: RUN 1 RUN 2 RUN 3 || RIGHT RIGHT 1\n\nExample 3:\nInstruction: turn around left twice and walk\nExplanation:\nBecause of the word \"and\", the instruction can be divided in two sub-instructions: 1) \"turn around left twice\" and 2) \"walk\"\n1) \"turn around left twice\". The word \"around\" results in 4 repetitions, therefore, \"turn around left\" corresponds to 4 LEFT commands. Because of the word \"twice\", \"turn around left twice\" results in 2 x 4 = 8 LEFT commands.\n2) \"walk\" results in 1 WALK command.\nOutput: LEFT 1 LEFT 2 LEFT 3 LEFT 4 LEFT 5 LEFT 6 LEFT 7 LEFT 8 || WALK 1\n\nExample 4:\nInstruction: walk twice after look around right\nExplanation:\nThe word \"after\" results in the inversion of the two sub-instructions 2) \"walk twice\" and 1) \"look around right\".\n1) \"look around right\". The word \"around\" results in 4 repetitions, therefore, \"look around right\" results in 4 RIGHT LOOK commands.\n2) \"walk twice\" results in 2 WALK commands.\nOutput: RIGHT LOOK 1 RIGHT LOOK 2 RIGHT LOOK 3 RIGHT LOOK 4 || WALK 1 WALK 2\n\nExample 5:\nInstruction: walk twice then look opposite left\nExplanation:\nBecause of the word \"then\", the instruction can be divided in two sub-instructions: 1) \"walk twice\" and 2) \"look opposite left\"\n1) \"walk twice\" results in 2 WALK commands.\n2) \"look opposite left\". The word \"opposite\", results in 2 repetitions of command LEFT, therefore, \"look opposite left\" results in 1 LEFT LEFT LOOK command.\nOutput: WALK 1 WALK 2 || LEFT LEFT LOOK 1\n\nExample 6:\nInstruction: run around left\nExplanation:\n1) \"run around left\". The word \"around\" results in 4 repetitions, therefore, \"run around left\" results in 4 LEFT RUN commands.\nOutput: LEFT RUN 1 LEFT RUN 2 LEFT RUN 3 LEFT RUN 4\n\nExample 7:\nInstruction: run around left thrice\nExplanation:\n1) \"run around left twice\". The word \"around\" results in 4 repetitions, therefore, \"run around left\" corresponds to 4 LEFT RUN commands. Because of the word \"thrice\", \"run around left thrice\" results in 3 x 4 = 12 LEFT RUN commands.\nOutput: LEFT RUN 1 LEFT RUN 2 LEFT RUN 3 LEFT RUN 4 LEFT RUN 5 LEFT RUN 6 LEFT RUN 7 LEFT RUN 8 LEFT RUN 9 LEFT RUN 10 LEFT RUN 11 LEFT RUN 12\n\nExample 8:\nInstruction: jump around right twice after run opposite left twice\nExplanation:\nThe word \"after\" results in the inversion of the two sub-instructions 2) \"jump around right twice\" and 1) \"run opposite left twice\".\n1) \"run opposite left twice\". The word \"opposite\", results in 2 repetitions of command LEFT, therefore, \"run opposite left\" corresponds to 1 LEFT LEFT RUN command. Because of the \"twice\", \"run opposite left twice\" results in 2 LEFT LEFT RUN commands.\n2) \"jump around right twice\". The word \"around\" results in 4 repetitions, therefore, \"jump around right\" corresponds to 4 RIGHT JUMP commands. Because of the word \"twice\", \"jump around right twice\" results in 2 x 4 = 8 RIGHT JUMP commands.\nOutput: LEFT LEFT RUN 1 LEFT LEFT RUN 2 || RIGHT JUMP 1 RIGHT JUMP 2 RIGHT JUMP 3 RIGHT JUMP 4 RIGHT JUMP 5 RIGHT JUMP 6 RIGHT JUMP 7 RIGHT JUMP 8\n\nExample 9:\nInstruction: turn around right twice\nExplanation:\n1) \"turn around right twice\". The word \"around\" results in 4 repetitions, therefore, \"turn around right\" corresponds to 4 RIGHT commands. Because of the word \"twice\", \"turn around left twice\" results in 2 x 4 = 8 RIGHT commands.\nOutput: RIGHT 1 RIGHT 2 RIGHT 3 RIGHT 4 RIGHT 5 RIGHT 6 RIGHT 7 RIGHT 8\n\n"

In [34]:
print(base_prompt)

Example 1:
Instruction: jump twice then turn left twice
Explanation:
Because of the word "then", the instruction can be divided in two sub-instructions: 1) "jump twice" and 2) "turn left twice"
1) "jump twice" results in 2 JUMP commands.
2) "turn left twice" results in 2 LEFT commands.
Output: JUMP 1 JUMP 2 || LEFT 1 LEFT 2

Example 2:
Instruction: turn opposite right after run thrice
Explanation:
The word "after", results in the inversion of the two sub-instructions 2) "turn opposite right" and 1) "run thrice".
1) "run thrice" results in 3 RUN commands.
2) "turn opposite right" . The word "opposite", results in 2 repetitions of command RIGHT, therefore, "turn opposite right" results in 1 RIGHT RIGHT command.
Output: RUN 1 RUN 2 RUN 3 || RIGHT RIGHT 1

Example 3:
Instruction: turn around left twice and walk
Explanation:
Because of the word "and", the instruction can be divided in two sub-instructions: 1) "turn around left twice" and 2) "walk"
1) "turn around left twice". The word "arou

In [35]:
test_samples = pd.read_csv('test_set_randon_400.tsv', sep='\t')

In [ ]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
responses = []

for idx, sample in enumerate(test_samples['instruction']):
    
  prompt_sample=base_prompt+'Example 10:\nInstruction: '+sample+'\nExplanation:'

  response = openai.Completion.create(
  engine="text-davinci-002",
  prompt=prompt_sample,
  temperature=0.0,
  max_tokens=356,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["\n\n"]
  )

  responses.append(response)

In [ ]:
responses[3]

<OpenAIObject text_completion id=cmpl-50bqQ4beUMQr2iFc6AxP2o4yJYCtU at 0x7f0f160bdef0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " \"look around right\" results in 4 RIGHT LOOK commands, and \"look around right thrice\" results in 3 RIGHT RIGHT LOOK commands.\nOutput: RIGHT LOOK 1 RIGHT LOOK 2 RIGHT LOOK 3 RIGHT LOOK 4 RIGHT RIGHT LOOK 1 RIGHT RIGHT LOOK 2 RIGHT RIGHT LOOK 3"
    }
  ],
  "created": 1650827158,
  "id": "cmpl-50bqQ4beUMQr2iFc6AxP2o4yJYCtU",
  "model": "text-davinci:002",
  "object": "text_completion"
}

In [41]:
!pip uninstall pandas
!pip install  pandas

Found existing installation: pandas 1.3.5
Uninstalling pandas-1.3.5:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/pandas-1.3.5.dist-info/*
    /usr/local/lib/python3.7/dist-packages/pandas/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/pandas/__init__.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_config/__init__.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_config/config.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_config/dates.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_config/display.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_config/localization.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_libs/__init__.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_libs/indexing.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_libs/interval.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_libs/missing.pyi
    /usr/local/lib/python3.7/dist-packages/pandas/_libs/pr

In [42]:
#save results
prompt = test_samples['instruction']
ground_truth=test_samples['output']
save_results(prompt, ground_truth, responses, '/outputs/result_rationales_markups.tsv')